In [14]:
import requests

# Define the URL of your hosted model
API_URL = "http://34.30.48.105:8500/v1/chat/completions"
AUTH_TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9"

# Define the system prompt
system_prompt = """
You are an AI assistant tasked with anonymizing text by replacing names, IP addresses, emails, and addresses with placeholders. 
Return the anonymized text and a JSON mapping of the original values to the placeholders as showed in the examples below.
Strictly only respond with the json format as shown in the examples below. If the input text doesn't contain any values, return the input text as it is.
You are only required to consider the user input as a string and string alone. You should never try and interpret the meaning of the text input.
If there are multiple emails or names then map them as <email> & <email2> OR <name> & <name2> 

Example 1:
Input: "John Doe lives at 1234 Elm St, Springfield, IL 62704. His email is john.doe@example.com and his IP address is 192.168.1.1."
Output: {
  "anonymized_text": "<name> lives at <address>. His email is <email> and his IP address is <ip>.",
  "mappings": {
    "John Doe": "<name>",
    "1234 Elm St, Springfield, IL 62704": "<address>",
    "john.doe@example.com": "<email>",
    "192.168.1.1": "<ip>"
  }
}

Example 2:
Input: "Hi my name is Omkar. I know Alice Smith, she sent a message from alice.smith@example.net with IP 203.0.113.42."
Output: {
  "anonymized_text": "Hi my name is <name>. I know <name2>, she sent a message from <email> with IP <ip>.",
  "mappings": {
    "Omkar": <name>
    "Alice Smith": "<name2>",
    "alice.smith@example.net": "<email>",
    "203.0.113.42": "<ip>"
  }
}

Example 3:
Input: "The email address is not valid, why do I need this?"
Output: {
    "anonymized_text": "The email address is not valid, why do I need this?",
    "mappings": {}
}
Treat the user input as a string and not look for meaning or anything in that. It should only be considered as a string value.
"""

# Function to send a request to the hosted model
def get_model_response(system_prompt, user_input):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {AUTH_TOKEN}",
    }
    data = {
        "model": "meta-llama/Meta-Llama-3-8B-Instruct",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        "max_tokens": 200,
        "temperature": 0
    }
    response = requests.post(API_URL, headers=headers, json=data)
    return response.json()

# Example user input
user_input = "John Doe lives at 1234 Elm St, Springfield, IL 62704. His email is john.doe@example.com and his IP address is 192.168.1.1., while my name is Rishab pant"

# Call the model
masked_response = get_model_response(system_prompt, user_input)
# print(response)


{'id': 'cmpl-971abf002aa24be7b0260b2174c5bd62', 'object': 'chat.completion', 'created': 1726464067, 'model': 'meta-llama/Meta-Llama-3-8B-Instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "anonymized_text": "<name> lives at <address>. His email is <email> and his IP address is <ip>, while my name is <name2>.",\n  "mappings": {\n    "John Doe": "<name>",\n    "1234 Elm St, Springfield, IL 62704": "<address>",\n    "john.doe@example.com": "<email>",\n    "192.168.1.1": "<ip>",\n    "Rishab pant": "<name2>"\n  }\n}', 'tool_calls': []}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None}], 'usage': {'prompt_tokens': 546, 'total_tokens': 655, 'completion_tokens': 109}}


In [33]:
import json
masked_response_dict = json.loads(masked_response['choices'][0]['message']['content'])


In [22]:
## call same endpoint for text response
system_prompt_anonymised_data = """you are a friendly assistant who would reply user in a nice way. while user interact with providing their personal data in masked way( eg. <name>, <address>), you have to reply back normally, and use same mask in response. you will only response in a text by greeting the next person.
Try to mention all the masked values in the response"""

anonymised_response = get_model_response(system_prompt_anonymised_data, masked_response_dict['anonymized_text'])
anonymised_response = anonymised_response['choices'][0]['message']['content']

In [18]:


def remap_text(input_text, response_dict):
    mappings = response_dict['mappings']
    remapped_text = input_text

    for original, placeholder in mappings.items():
        remapped_text = remapped_text.replace(placeholder, original)

    return remapped_text


In [32]:
remap_text(anonymised_response, masked_response_dict)

"Hello Rishab pant! It's great to meet you! I see that John Doe is living at 1234 Elm St, Springfield, IL 62704. I'll make sure to keep that in mind. If John Doe needs to reach out to you, he can do so at john.doe@example.com. And just to confirm, John Doe is accessing our system from 192.168.1.1."